# 线性回归 Linear Regression

## 线性模型

自变量<b>x</b>跟因变量y之间的关系时线性的，即y可表示为<b>x</b>中元素的加权和。线性建模：

$$ \hat{y} = \mathbf{w}^\top \mathbf{x} + b $$

每个样本含有 d 个 feature, 我们的数据集有 n 个样本，则可以用 n x d 矩阵 $\mathbf{X}$ 表示整个数据集的所有特征，向量 $\hat{\mathbf{y}} \in \mathbb{R}^n$ 表示所有的预测结果：

$$ {\hat{\mathbf{y}}} = \mathbf{X} \mathbf{w} + b $$

## 损失函数 Loss function

损失函数量化目标的实际值与预测值之间的差距。回归问题中最常用的损失函数是平方误差函数 `MSE` 。当样本 i 的预测值为 $ \hat{y}^{(i)} $，其对应的实际标签为 $ y^{(i)} $ 时，该样本的平方误差 $ l^{(i)} $ 可定义为 （常数 1/2 是为了方便求导计算梯度）：

$$ l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2 $$

为了评价模型在整改数据集上的表现，我们计算它在整个数据集 n 个样本上的平方误差的平均值：

$$ L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2 $$

在训练模型时，我们希望寻找到一组参数 $ \mathbf{w}^*, b^* $ 来最小化在整个数据集 n 个样本上的总损失平均值（即上面的损失函数） ：

$$ \mathbf{w}^*, b^* = \operatorname*{argmin}_{\mathbf{w}, b}\  L(\mathbf{w}, b) $$

## 随机梯度下降 SGD (Stochastic Gradient Descent)

随机梯度下降几乎可以优化所有深度学习模型，它通过不断地在损失函数递减的方向上更新参数来持续减少误差。

SGD 最简单的实现方法是计算损失函数（数据集中所有样本的损失均值）关于模型参数的导数（梯度），然后向梯度反方向更新参数。

每次更新参数之前我们需要遍历整个数据集，在现实中这样做太慢，所以我们通常在每次需要计算更新的时候随机抽取一小批样本，这种做法叫 `小批量随机梯度下降 (minibatch stochastic gradient descent)`。



进行 SGD 时，我们先随机初始化所有参数 $ (\mathbf{w},b) $，然后迭代更新它们。

每次迭代，

1. 先随机抽取一个小批量 $ \mathcal{B} $（批量中的样本数是一个由我们选择的固定的超参数）
2. 然后用 Loss function 计算模型对这个小批量的样本进行预测的平均损失的偏导数得到梯度 $ \partial_{(\mathbf{w},b)} $
3. 再将这个梯度乘以学习率 $ \eta $（这是另一个由我们决定的固定的超参数）得到每个参数的更新量
4. 最后从参数中减去更新量完成这一次更新。

表示成式子如下：

$$ (\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b) $$

线性回归的 Loss 函数很好计算：

$$ \begin{split}\begin{aligned} \mathbf{w} &\leftarrow \mathbf{w} -   \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{\mathbf{w}} l^{(i)}(\mathbf{w}, b) = \mathbf{w} - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \mathbf{x}^{(i)} \left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right),\\ b &\leftarrow b -  \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_b l^{(i)}(\mathbf{w}, b)  = b - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right). \end{aligned}\end{split} $$

我们根据迭代结果来调整超参数，称为调参 hyperparameter tuning。


利用 torch 等的线性代数计算我们可以同时处理整个小批量的样本来提高效率，所以我们对计算进行矢量化。

线性回归可以看作是单层神经网络。

## 从零开始实现线性回归

In [3]:
!pip3 install matplotlib torch d2l

     |████████████████████████████████| 7.3 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 68.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 30.1 MB/s eta 0:00:01
     |████████████████████████████████| 229 kB 77.9 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 16.0 MB/s  eta 0:00:01
     |████████████████████████████████| 98 kB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 127.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 108.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.2 MB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 53.5 MB/s  eta 0:00:01
     |████████████████████████████████| 12.4 MB 59.1 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB

In [ ]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l